In [1]:
import pandas as pd
import numpy as np
import os
import sys

sys.path.append('../')

from src.utilities.config_ import combined_data_path, scrape_data_path, predicted_data_path
import src.utilities.utils as utils

In [2]:
df = utils.load(os.path.join(combined_data_path, "combined_data.feather"))
df

,title,date,url,category,label,source
0,"Gold, Silver Q3 Technical Forecast: Gold's Ran...",30-06-2024,https://www.dailyfx.com/analysis/gold-silver-q...,forex,neutral,dailyfx
1,Bitcoin Q3 Fundamental Outlook – Short-Term Mu...,30-06-2024,https://www.dailyfx.com/news/bitcoin-q3-fundam...,forex,positive,dailyfx
2,Crude Oil Q3 Technical Forecast: Narrowing Pri...,29-06-2024,https://www.dailyfx.com/analysis/crude-oil-q3-...,forex,neutral,dailyfx
3,Equities Q3 Fundamental Outlook: Bullish Momen...,29-06-2024,https://www.dailyfx.com/news/equities-q3-funda...,forex,neutral,dailyfx
4,British Pound Q3 Technical Forecast: GBP/USD E...,29-06-2024,https://www.dailyfx.com/analysis/british-pound...,forex,neutral,dailyfx
...,...,...,...,...,...,...
1044,"Increase in exports, improvement in CAD, mnfg ...",30-06-2024,https://economictimes.indiatimes.com//news/eco...,economy,positive,econtimes
1045,India initiates anti-dumping probe into import...,30-06-2024,https://economictimes.indiatimes.com//news/eco...,economy,neutral,econtimes
1046,2015 skilling policy to be replaced with new one,30-06-2024,https://economictimes.indiatimes.com//news/eco...,economy,neutral,econtimes
1047,India is witnessing high foreign remittances &...,30-06-2024,https://economictimes.indiatimes.com//news/eco...,economy,positive,econtimes


In [3]:
# filter based on source
dailyfx_df = df[df["source"] == "dailyfx"].reset_index(drop=True)
econtimes_df = df[df["source"] == "econtimes"].reset_index(drop=True)
financialtimes_df = df[df["source"] == "financialtimes"].reset_index(drop=True)

In [10]:
def get_sentiment_stats(df):
    # Get the total count of each sentiment label
    label_counts = df['label'].value_counts()

    # get total for each label 
    total_neutral = label_counts.get('neutral', 0)
    total_positive = label_counts.get('positive', 0)
    total_negative = label_counts.get('negative', 0)

    # return
    total_negative, total_neutral, total_positive

    try:
        score = (total_positive - total_negative) / (total_negative + total_positive + total_neutral)

    except ZeroDivisionError as e:
        score = 0

    print(f'Score: {score}')
    return score

def calculate_sentiment_metrics(df):

    # count label
    label_counts = df['label'].value_counts()

    # get total for each label 
    total_neutral = label_counts.get('neutral', 0)
    total_positive = label_counts.get('positive', 0)
    total_negative = label_counts.get('negative', 0)
    
    # Weighted Sentiment Score (weights can be adjusted as needed)
    w_p = .75
    w_n = .75
    weighted_sentiment_score = (((w_p * total_positive) - (w_n * total_negative)) / (total_positive + total_negative + total_neutral))

    return weighted_sentiment_score
    
for j in range(1, 31):
    formatted_number = f"{j:02}"
    score = calculate_sentiment_metrics(df[df["date"] == f"{formatted_number}-06-2024"])
    print(f"Weighted NSS: {score}")

Weighted NSS: 0.3088235294117647
Weighted NSS: 0.1125
Weighted NSS: 0.11029411764705882
Weighted NSS: -0.12
Weighted NSS: -0.018292682926829267
Weighted NSS: 0.0234375
Weighted NSS: 0.055970149253731345
Weighted NSS: 0.10714285714285714
Weighted NSS: 0.375
Weighted NSS: -0.125
Weighted NSS: 0.15789473684210525
Weighted NSS: -0.038135593220338986
Weighted NSS: 0.06382978723404255
Weighted NSS: 0.1377551020408163
Weighted NSS: 0.1875
Weighted NSS: 0.2
Weighted NSS: 0.125
Weighted NSS: 0.2215909090909091
Weighted NSS: 0.0625
Weighted NSS: 0.11413043478260869
Weighted NSS: 0.024193548387096774
Weighted NSS: 0.13043478260869565
Weighted NSS: 0.09375
Weighted NSS: 0.04891304347826087
Weighted NSS: 0.01744186046511628
Weighted NSS: 0.036585365853658534
Weighted NSS: 0.06976744186046512
Weighted NSS: 0.16216216216216217
Weighted NSS: 0.028846153846153848
Weighted NSS: 0.19736842105263158
